In [1]:
import netCDF4 as nc
import os
import numpy as np
import pandas as pd

## Нужная переменная zos (sea_surface_height_above_geoid)

In [3]:
test_file='C:/Users/Julia/Documents/ITMO/Master_degree_research/WM_folder/SPITZ_1d_00010101_00011231_grid_T_00010101-00010114.nc'
ds = nc.Dataset(test_file)
for var in ds.variables.values():
    print(var)
print(ds['zos'])

<class 'netCDF4._netCDF4.Variable'>
float32 nav_lat(y, x)
    standard_name: latitude
    long_name: Latitude
    units: degrees_north
unlimited dimensions: 
current shape = (115, 117)
filling off
<class 'netCDF4._netCDF4.Variable'>
float32 nav_lon(y, x)
    standard_name: longitude
    long_name: Longitude
    units: degrees_east
unlimited dimensions: 
current shape = (115, 117)
filling off
<class 'netCDF4._netCDF4.Variable'>
float32 deptht(deptht)
    name: deptht
    long_name: Vertical T levels
    units: m
    positive: down
    bounds: deptht_bounds
unlimited dimensions: 
current shape = (75,)
filling off
<class 'netCDF4._netCDF4.Variable'>
float32 deptht_bounds(deptht, axis_nbounds)
    units: m
unlimited dimensions: 
current shape = (75, 2)
filling off
<class 'netCDF4._netCDF4.Variable'>
float64 time_centered(time_counter)
    standard_name: time
    long_name: Time axis
    calendar: noleap
    units: seconds since 1900-01-01 00:00:00
    time_origin: 1900-01-01 00:00:00
    b

In [5]:
# с координатами то че делать
# переменную time нужно как-то обработать (перевести секунды в норм формат)
# не забыть что в каждом файле 14 дней (14 значений по переменной zos-time нужно извлечь)